# Chapter 3: Negative Prompts

---

**Lesson:**

Very similar to building strong prompts is crafting strong negative prompts. The purpose here is to indicate what you *don't* want to see in your image. We do this by providing additional prompts that follow the same format and concept as "positive" prompts, but the purpose is *opposite*.

In [ ]:
%pip install ipython

import json
import boto3
from PIL import Image
import base64
import io
from random import randint
from IPython.display import display

def generate_image(text):
    # Create a new BedrockRuntime client.
    bedrock_runtime = boto3.client(
        "bedrock-runtime",
        region_name="us-east-1",
    )
    
    # Configure the inference parameters.
    inference_params = {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": text,
            "negativeText": negative_text,
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "quality": "premium",
            "height": 1024,
            "width": 1024,
            "cfgScale": 8.0,
            "seed": randint(0, 3000),  # Use a random seed
        },
    }
    
    # Invoke the model.
    response = bedrock_runtime.invoke_model(
        modelId="amazon.titan-image-generator-v1", body=json.dumps(inference_params)
    )
    
    # Convert the JSON string to a dictionary.
    response_body = json.loads(response["body"].read())
    
    # Loop through the generated images and display each in the notebook.
    images = response_body["images"]
    for i in range(len(images)):
        image_data = images[i]
        image_bytes = base64.b64decode(image_data)
        image = Image.open(io.BytesIO(image_bytes))
        display(image)

## How to add a negative prompt and why

It should be noted that images can still be generated even without a negative prompt (else we wouldn't have created anything of value in our previous chapters). However, we can enhance our images by clarifying what we don't want. The reason this is important can be exemplified by the below.


In [ ]:
text = "a person without a hat"
negative_text = "n/a"
generate_image(text)

Intuitively, we would expect the above prompt to generate an image of a person that is not wearing a hat.

What happened here? We specified a person *without* a hat, and yet that is not what we got. Herein lies the importance of negative prompting: Amazon Titan Image Generator G1 *expects* us to use negative prompting to specify what we do not want in our image. There is no promise that a given element (such as a hat) will appear in your generated image, but to truly eliminate or at least greatly reduce the probability of that element appearing, we must use negative prompts.

Now, let's try the above example but with negative prompts.

In [ ]:
text = "a person"
negative_text = "hat"
generate_image(text)

## Example:

**Example 3.1 - A positive image**

Let's experiment more with negative prompts by creating a crisp image that changes with every generation (i.e., seed is 0), but remains sunny without a cloud in the sky.

In [ ]:
text = "a sunny day, green grass, smiling person, crisp image"
negative_text = "clouds, storms, sad"
generate_image(text)

## Exercises:

We will now incorporate negative prompting into some previous examples to see how the outcome is affected.

**Exercise 3.1 - Replicating stories**

Using proper formatting, the included parameters, detailed descriptions, and negative prompting, generate an image based a scene in a book you read or a movie you saw recently.

In [ ]:
text = "A dragon flying over a midieval town while the people of the town run away."
negative_text = "river, cinematic"
generate_image(text)

**Exercise 3.2 - Capturing an image**

See if you can recreate an image of any particular scene in your life, be it your office, park, house, etc. Think about the details required to make this come to life.

In [ ]:
text = "Skydiving in the Swiss Alps with mountains below as well as greenery and a huge lake in the middle."
negative_text = "forests, clouds, desert, cinematic"
generate_image(text)

# Chapter 3 - END